In [3]:
import sqlite3
import pandas as pd
import os
import geopandas as gpd
from shapely.geometry import Point
import osmnx as ox

conn = sqlite3.connect("my_database_project.db")
cur = conn.cursor()

In [4]:

csv_file_path = 'Datensätze\Bezirke_Durchschnitt_wetter.csv' 
# Load the data from csv
df = pd.read_csv(csv_file_path, delimiter=',', encoding ='latin1')
df[df.columns[0]] = pd.to_datetime(df.iloc[:, 0], errors='coerce')
df['Date'] = df.iloc[:,0].dt.strftime('%d.%m.%Y')  # Extract the date part
df['TimeID'] = df.iloc[:,0].dt.hour  # Extract the time part
df.reset_index()

date_dim = pd.read_sql_query("SELECT DateID, Date FROM Date_dim", conn)
df = pd.merge(df,  date_dim[['DateID', 'Date']], on='Date', how='left')
df.drop(['time','is_day ()','Date'], inplace= True,axis=1)
new_order = ['DateID', 'TimeID','temperature_2m (°C)', 'relative_humidity_2m (%)', 'rain (mm)',
       'snowfall (cm)', 'cloud_cover (%)', 'wind_speed_10m (km/h)', 'sunshine_duration (s)']  # Specify the new order
df = df[new_order]


<>:1: SyntaxWarning: invalid escape sequence '\B'
<>:1: SyntaxWarning: invalid escape sequence '\B'
C:\Users\user\AppData\Local\Temp\ipykernel_37452\2272774239.py:1: SyntaxWarning: invalid escape sequence '\B'
  csv_file_path = 'Datensätze\Bezirke_Durchschnitt_wetter.csv'


In [5]:
df

,DateID,TimeID,temperature_2m (°C),relative_humidity_2m (%),rain (mm),snowfall (cm),cloud_cover (%),wind_speed_10m (km/h),sunshine_duration (s)
0,1,0,11.066667,71.666667,0.000000,0.0,61.333333,23.8,0.0
1,1,1,11.141667,70.000000,0.016667,0.0,77.250000,25.3,0.0
2,1,2,11.591667,64.500000,0.000000,0.0,92.416667,29.0,0.0
3,1,3,11.825000,62.083333,0.000000,0.0,95.916667,28.8,0.0
4,1,4,11.641667,63.416667,0.000000,0.0,93.916667,27.2,0.0
...,...,...,...,...,...,...,...,...,...
52579,2191,19,6.341667,82.916667,0.141667,0.0,100.000000,17.1,0.0
52580,2191,20,6.366667,82.500000,0.050000,0.0,100.000000,17.0,0.0
52581,2191,21,6.725000,83.583333,0.000000,0.0,99.750000,15.7,0.0
52582,2191,22,6.066667,85.916667,0.000000,0.0,90.333333,14.3,0.0


In [6]:

# Insert data into the table
df.to_sql('Wetter', conn, if_exists = 'append', index = False)

conn.commit()

In [7]:
df = pd.read_sql_query('SELECT * FROM Wetter', conn)
df

,DateID,TimeID,temperature_2m (°C),relative_humidity_2m (%),rain (mm),snowfall (cm),cloud_cover (%),wind_speed_10m (km/h),sunshine_duration (s)
0,1,0,11.066667,71.666667,0.000000,0.0,61.333333,23.8,0.0
1,1,1,11.141667,70.000000,0.016667,0.0,77.250000,25.3,0.0
2,1,2,11.591667,64.500000,0.000000,0.0,92.416667,29.0,0.0
3,1,3,11.825000,62.083333,0.000000,0.0,95.916667,28.8,0.0
4,1,4,11.641667,63.416667,0.000000,0.0,93.916667,27.2,0.0
...,...,...,...,...,...,...,...,...,...
52579,2191,19,6.341667,82.916667,0.141667,0.0,100.000000,17.1,0.0
52580,2191,20,6.366667,82.500000,0.050000,0.0,100.000000,17.0,0.0
52581,2191,21,6.725000,83.583333,0.000000,0.0,99.750000,15.7,0.0
52582,2191,22,6.066667,85.916667,0.000000,0.0,90.333333,14.3,0.0


In [ ]:
cur.close()
conn.close()